In [10]:
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained model and tokenizer
model_name = 'gpt2'  # You can choose a different model if desired
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
letemcook = GPT2LMHeadModel.from_pretrained(model_name)

# Set the model to evaluation mode
letemcook.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [53]:
# Function to generate text completion
def generate_completion(model, prompt, max_length=20):
    # Encode the input prompt
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    # Generate output
    with torch.no_grad():
        outputs = model.generate(inputs, max_new_tokens=max_length, min_length = 3, temperature=0.99, do_sample=True)

    # Decode the generated output
    completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return completion

def autocomplete_recipe_step(model, title, ingredients, complete_instruction_steps, incomplete_instruction_steps, print_prompt = False):
    pre_prompt = "You are a chef-bot autocompleting a small part of a recipe: [START_OF_RECIPE] "
    title_prompt = f"[RECIPE_TITLE] {title} " 
    ingredients_prompt = f"[INGREDIENTS_LIST] {ingredients} " 
    
    instructions_prompt = f"[STEPS] "
    for i, step in enumerate(complete_instruction_steps):
        instructions_prompt += f"{i + 1} - {step} "
    instructions_prompt += " {DO NOT CREATE ANOTHER STEP AFTER STEP 3} " + f"{len(complete_instruction_steps) + 1} - {incomplete_instruction_steps}"
    
    # prompt = pre_prompt + title_prompt + ingredients_prompt + instructions_prompt
    prompt = pre_prompt + instructions_prompt
    if print_prompt: print(prompt)
    return generate_completion(model, prompt)


def convert_string_to_list(input_string):
    clean_string = input_string.strip('[]')
    return [s.strip().strip('"') for s in clean_string.split('", "')] # assuming this is always a good split

def index_of_instruction_word(text, n):
    words = text.split()
    
    if n > len(words):
        return 0    
    
    position = 0
    for word in words[:n-1]:
        position += len(word) + 1 
    
    return position


from IPython.display import display, HTML
def display_prompt(text):
    display(HTML(f"<h1>Prompt</h1><p>{text}</p>"))

In [54]:
# Change nrows if desired
df = pd.read_csv('NLG_subset.csv', nrows=5)

In [ ]:
def separate_recipe_components(recipe_index, n_complete_steps=2, n_words_before_autocomplete=3):

    test_rec = df.iloc[recipe_index, :]

    title = test_rec["title"]
    ingredients = test_rec["ingredients"]

    steps = convert_string_to_list(test_rec["directions"])
    complete_steps = steps[:n_complete_steps - 1]

    if len(steps) < n_complete_steps: return title, ingredients, complete_steps, steps[len(steps)-1], steps[len(steps)-1]

    true_step = steps[n_complete_steps]
    n_characters_before_autocomplete = index_of_instruction_word(true_step, n_words_before_autocomplete) # Not used for now

    n_characters_before_autocomplete = 6 # Hardcoded for now
    incomplete_instruction_step = true_step[0:n_characters_before_autocomplete]
    

    #print(f"Autocompleting the instruction step starting with: {incomplete_instruction_step}")
    #print(f"TRUE STEP: {true_step}")
    
    return title, ingredients, complete_steps, incomplete_instruction_step, true_step


In [ ]:
recipes = df.index.tolist()
results = []

for recipe_index in recipes:

    # Mess around with n_complete_steps and n_words_before_autcomplete    
    title, ingredients, complete_steps, incomplete_instruction_step, true_step = separate_recipe_components(recipe_index, n_complete_steps=2, n_words_before_autocomplete=4)
    autocompleted_step = autocomplete_recipe_step(letemcook, title, ingredients, complete_steps, incomplete_instruction_step)
    
    results.append((incomplete_instruction_step, true_step, autocompleted_step))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [65]:
for result in results:
    print("-" * 80)
    print(f"Autocompleting the instruction step starting with: {result[0]}")
    print(f"TRUE STEP: {result[1]}")
    display_prompt(result[2])

--------------------------------------------------------------------------------
Autocompleting the instruction step starting with: Boil a
TRUE STEP: Boil and stir 5 minutes more. Take off heat.


--------------------------------------------------------------------------------
Autocompleting the instruction step starting with: Mix so
TRUE STEP: Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\u00b0 for 3 hours.


--------------------------------------------------------------------------------
Autocompleting the instruction step starting with: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.
TRUE STEP: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.


--------------------------------------------------------------------------------
Autocompleting the instruction step starting with: Pour g
TRUE STEP: Pour gravy and cream of mushroom soup over chicken; level.


--------------------------------------------------------------------------------
Autocompleting the instruction step starting with: Keep i
TRUE STEP: Keep in refrigerator.
